In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

In [11]:
## Download checkpoint for base 8x8 model
model_checkpoint_dir = os.path.expanduser('../../CounterfactualWorldModels/checkpoints')
if not os.path.exists(model_checkpoint_dir):
    os.makedirs(model_checkpoint_dir)
    
## download base 8x8 2-frame model
base8x8_url = 'https://counterfactual-world-modeling.s3.amazonaws.com/cwm_baseVMAE_224px_8x8patches_2frames.pth'
!wget -nc --show-progress -P $model_checkpoint_dir $base8x8_url

File ‘../../CounterfactualWorldModels/checkpoints/cwm_baseVMAE_224px_8x8patches_2frames.pth’ already there; not retrieving.



In [19]:
torch.backends.mps.is_available()

True

In [23]:
import cwm.models.VideoMAE.vmae as vmae

DEVICE = torch.device('cpu')
if torch.cuda.is_available():
    DEVICE = torch.device('cuda:0')
elif torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    
USE_FLASH = 'cuda' in str(DEVICE.type)

model_func = vmae.base_8x8patch_2frames_1tube
model_path = os.path.join(model_checkpoint_dir, base8x8_url.split('/')[-1])

model = vmae.base_8x8patch_2frames_1tube(use_flash_attention=(USE_FLASH))
model.eval().requires_grad_(False).to(DEVICE)
print("Num parameters: %d" % sum([v.numel() for v in model.parameters()]))

did_load = model.load_state_dict(torch.load(model_path, map_location='cpu')['model'])
print(did_load, model_func.__name__, model_path, DEVICE)

NUM PATCHES IN ENCODER 1568
Num parameters: 92661312
<All keys matched successfully> base_8x8patch_2frames_1tube ../../CounterfactualWorldModels/checkpoints/cwm_baseVMAE_224px_8x8patches_2frames.pth mps


In [24]:
model.mask_size

(2, 28, 28)

In [39]:
x = torch.rand((1,3,2,224,224)).to(device)
N = np.prod(model.mask_size)
mask = torch.cat([torch.zeros((1, N//2)), torch.ones((1, N//2))], -1).bool().to(x.device)

model = model.to(device)

y = model(x, mask)

RuntimeError: Conv3D is not supported on MPS

In [38]:
model.encoder.patch_embed.proj.weight.shape

torch.Size([1024, 3, 1, 4, 4])

In [4]:
a = torch.rand((10)).to('mps')
b = torch.lerp(a, a, a)

/var/folders/z8/fzyc5zw96k3glmjvn5ctc8sh0000gn/T/ipykernel_49535/575506171.py:2: UserWarning: The operator 'aten::lerp.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  b = torch.lerp(a, a, a)
